In [ ]:
import aiohttp
import aiohttp.web
import asyncio

In [ ]:
async def handle(request):
    name = request.match_info.get("name", "Anonymous")
    text = f"Hello, {name}"
    return aiohttp.web.Response(text=text)

In [ ]:
event = asyncio.Event()


async def stop(request):
    event.set()


app = aiohttp.web.Application()
app.add_routes(
    [
        aiohttp.web.get("/", handle),
        aiohttp.web.get("/stop", stop),
        aiohttp.web.get("/{name}", handle),
    ]
)

runner = aiohttp.web.AppRunner(app)
await runner.setup()
site = aiohttp.web.TCPSite(runner, "localhost", 43210)
await site.start()

_ = await event.wait()

In [ ]:
async def client():
    async with aiohttp.ClientSession() as session:
        async with session.get("http://localhost:43210/") as response:
            text = await response.text()
    return text


await client()

In [ ]:
async with aiohttp.ClientSession() as session:
    try:
        await session.get("http://localhost:43210/stop")
    except aiohttp.ServerDisconnectedError:
        pass